In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.linear_model import LinearRegression
from datetime import datetime

In [2]:
db_path = './data/cruise_pre.db'
conn = sqlite3.connect(db_path)
cruise_pre_data = pd.read_sql_query(sql="SELECT * FROM cruise_pre", con=conn)
# cruise_pre_data.head()
cruise_pre_data = cruise_pre_data.drop(['index','Logging'], axis=1)
cruise_pre_data.shape

(133746, 17)

In [3]:
db_path = './data/cruise_post.db'
conn = sqlite3.connect(db_path)
cruise_post_data = pd.read_sql_query(sql="SELECT * FROM cruise_post", con=conn)
# cruise_post_data.head()
cruise_post_data = cruise_post_data.drop(['index'], axis=1)
cruise_post_data.shape

(133746, 7)

In [4]:
# Fill the empty field for rows with same "Ext_Intcode" using forward and backwards fill
# This is to facilitate the dropping of duplicate rows

cruise_pre_data = cruise_pre_data.set_index('Ext_Intcode').groupby("Ext_Intcode").fillna(method="ffill").reset_index()
cruise_pre_data = cruise_pre_data.set_index('Ext_Intcode').groupby("Ext_Intcode").fillna(method="bfill").reset_index()


cruise_post_data = cruise_post_data.set_index('Ext_Intcode').groupby("Ext_Intcode").fillna(method="ffill").reset_index()
cruise_post_data = cruise_post_data.set_index('Ext_Intcode').groupby("Ext_Intcode").fillna(method="bfill").reset_index()


In [5]:
# # Check for cruise_pre duplicate
# duplicate_pre_rows = cruise_pre_data[cruise_pre_data.duplicated()]

# # # Shows cruise_pre duplicate
# print("Duplicate Rows (excluding first occurrence):")
# print(duplicate_pre_rows)

# Remove cruise_pre duplicate rows
cruise_pre_data = cruise_pre_data.drop_duplicates()
print("Shape of cruise_pre_data after droping duplicate:")
cruise_pre_data.shape

Number of rows and columns for pre_cruise after droping duplicate:


(129850, 17)

In [6]:
# # Check for cruise_post duplicate
# duplicate_post_rows = cruise_post_data[cruise_post_data.duplicated()]

# # # Shows cruise_post duplicate
# print("Duplicate Rows (excluding first occurrence):")
# print(duplicate_post_rows)

# Remove cruise_post duplicate rows
cruise_post_data = cruise_post_data.drop_duplicates()
print("Shape of cruise_post_data after droping duplicate:")
cruise_post_data.shape

Number of rows and columns for post_cruise after droping duplicate:


(129850, 7)

In [7]:
# Perform the join operation on the 'Ext_Intcode' column
cruise_merge_data = pd.merge(cruise_pre_data, cruise_post_data, on='Ext_Intcode', how='inner')
cruise_merge_data.head()
print("Shape of cruise_merge_data after merging:")
cruise_merge_data.shape

# Save the resulting DataFrame to a CSV file
cruise_merge_data.to_csv('cruise_merge.csv', index=False)


Number of rows and columns for after merging cruise_post and cruise_pre:


(129850, 23)

In [26]:
cruise_merge_data = pd.read_csv('cruise_merge.csv')

# Define a function to convert the values to integers
def convert_to_integer(value):
    if value is not np.nan and isinstance(value, str):
        parts = value.split()
        if len(parts) == 2:
            numeric_part, unit = parts
            if unit == "KM":
                return int(numeric_part)
            elif unit == "Miles":
                # Convert Miles to KM (1 Mile = 1.60934 KM)
                return int(float(numeric_part) * 1.60934)
    return np.nan

# Apply the function to the DataFrame column
cruise_merge_data['Cruise Distance'] = cruise_merge_data['Cruise Distance'].apply(convert_to_integer)

cruise_merge_data.head()

Ext_Intcode  Gender Date of Birth         Source of Traffic  \
0  LB446RWOOZI     NaN    05/10/1973  Direct - Company Website   
1  LB138HKBECM  Female           NaN   Indirect - Social Media   
2  BL713UHBAAN  Female    22/07/1998  Indirect - Search Engine   
3  LB243DMKCFL  Female    01/05/1970  Direct - Company Website   
4  LB218CFLOBS    Male    07/01/1960  Direct - Company Website   

   Onboard Wifi Service  Embarkation/Disembarkation time convenient  \
0    A little important                                         3.0   
1  Not at all important                                         4.0   
2                   NaN                                         3.0   
3        Very important                                         4.0   
4    Somewhat important                                         4.0   

   Ease of Online booking  Gate location Onboard Dining Service  \
0                     5.0            3.0         Very important   
1                     1.0            NaN         Very important   
2                     0.0            5.0                    NaN   
3                     4.0            4.0     Somewhat important   
4                     2.0            NaN   Not at all important   

   Online Check-in  ...  Baggage handling Port Check-in Service  \
0              2.0  ...               2.0                   4.0   
1              NaN  ...               3.0                   4.0   
2              NaN  ...               1.0                   2.0   
3              4.0  ...               NaN                   3.0   
4              2.0  ...               3.0                   5.0   

   Onboard Service  Cleanliness  Cruise Name  Ticket Type  Cruise Distance  \
0              2.0          3.0    Blastoise          NaN           3567.0   
1              4.0          4.0    Blastoise       Deluxe            672.0   
2              3.0          NaN       IAPRAS       Deluxe           1167.0   
3              2.0          4.0       Lapras       Deluxe            280.0   
4              2.0          1.0       Lapras     Standard           1842.0   

  WiFi Dining  Entertainment  
0  1.0      1            1.0  
1  NaN      0            1.0  
2  NaN      0            0.0  
3  NaN      0            1.0  
4  NaN      1            NaN  

[5 rows x 23 columns]

In [27]:
# Convert the "date_text" column to datetime format
cruise_merge_data['Date of Birth']= pd.to_datetime(cruise_merge_data['Date of Birth'])

# Specify date formats explicitly for parsing
date_formats = ['%Y-%m-%d', '%d/%m/%Y']

# Convert the "date_of_birth" column to datetime, handling multiple formats
cruise_merge_data['Date of Birth'] = pd.to_datetime(cruise_merge_data['Date of Birth'], format= date_formats, errors='coerce')


# Get the current date
current_date = datetime.now()

# Calculate the age and create an "age" column
cruise_merge_data['age'] = (current_date - cruise_merge_data['Date of Birth']).astype('<m8[Y]')

cruise_merge_data.head()

# Save the resulting DataFrame to a CSV file
# cruise_merge_data.to_csv('cruise_merge_age.csv', index=False)

# Drop irrelavant and dependent column
cruise_merge_data.drop(['Ext_Intcode','Date of Birth' ,'Ticket Type'],axis=1, inplace=True)
cruise_merge_data.shape


C:\Users\thedi\AppData\Local\Temp/ipykernel_17828/592375043.py:2: UserWarning: Parsing '22/07/1998' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cruise_merge_data['Date of Birth']= pd.to_datetime(cruise_merge_data['Date of Birth'])
C:\Users\thedi\AppData\Local\Temp/ipykernel_17828/592375043.py:2: UserWarning: Parsing '23/12/1986' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cruise_merge_data['Date of Birth']= pd.to_datetime(cruise_merge_data['Date of Birth'])
C:\Users\thedi\AppData\Local\Temp/ipykernel_17828/592375043.py:2: UserWarning: Parsing '20/03/1987' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cruise_merge_data['Date of Birth']= pd.to_datetime(cruise_merge_data['Date of Birth'])
C:\Users\thedi\AppData\Local\Temp/ipykernel_17828/592375043.py:2: UserWarning: Parsing '20/12/1981' in DD/MM/YYYY format. Provide format

C:\Users\thedi\AppData\Local\Temp/ipykernel_17828/592375043.py:2: UserWarning: Parsing '17/12/2001' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cruise_merge_data['Date of Birth']= pd.to_datetime(cruise_merge_data['Date of Birth'])
C:\Users\thedi\AppData\Local\Temp/ipykernel_17828/592375043.py:2: UserWarning: Parsing '21/03/1995' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cruise_merge_data['Date of Birth']= pd.to_datetime(cruise_merge_data['Date of Birth'])
C:\Users\thedi\AppData\Local\Temp/ipykernel_17828/592375043.py:2: UserWarning: Parsing '15/04/1968' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cruise_merge_data['Date of Birth']= pd.to_datetime(cruise_merge_data['Date of Birth'])
C:\Users\thedi\AppData\Local\Temp/ipykernel_17828/592375043.py:2: UserWarning: Parsing '16/04/2010' in DD/MM/YYYY format. Provide format

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
cruise_merge_data.dtypes

Gender                                         object
Source of Traffic                              object
Onboard Wifi Service                           object
Embarkation/Disembarkation time convenient    float64
Ease of Online booking                        float64
Gate location                                 float64
Onboard Dining Service                         object
Online Check-in                               float64
Cabin Comfort                                 float64
Onboard Entertainment                          object
Cabin service                                 float64
Baggage handling                              float64
Port Check-in Service                         float64
Onboard Service                               float64
Cleanliness                                   float64
Cruise Name                                    object
Cruise Distance                               float64
WiFi                                          float64
Dining                      